In [9]:
%load_ext autoreload
%autoreload 2

from typing import Union

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import datetime
import matplotlib.dates as mdates

from datetime import timedelta, date
import time
import os

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

from jax import random
import numpyro
import numpyro.distributions as dist

import scenario_generator.mcmc_forecast as mcmc
import scenario_generator.utils as u

from mechafil.data import get_historical_network_stats, get_sector_expiration_stats
from mechafil.power import forecast_power_stats, build_full_power_stats_df, scalar_or_vector_to_vector
from mechafil.vesting import compute_vesting_trajectory_df
from mechafil.minting import compute_minting_trajectory_df
from mechafil.supply import forecast_circulating_supply_df

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
NDEVICES = 2 
numpyro.set_host_device_count(NDEVICES)

# Data starts in filecoin_daily_stats 2021-03-15
# genesis block was in 2020-08-24
# Main net launch was in 2020-10-15

today = datetime.date(2023, 2, 12) 
DATA_LAG_DAYS = 0
current_date = today - datetime.timedelta(days=(DATA_LAG_DAYS))
current_day = (current_date - datetime.date(2020, 10, 15)).days

start_date = datetime.date(2023, 1, 1)
start_day = (start_date - datetime.date(2020, 10, 15)).days

# Forecast is one year ahead from today
forecast_lenght = 365 * 3
end_day = current_day + forecast_lenght
end_date = current_date + datetime.timedelta(days=forecast_lenght)

print(start_date)
print(current_date)
print(end_date)
duration = 365 # sector duration

# handy constants
EIB = 2 ** 60
PIB = 2 ** 50
TIB = 2 ** 40
GIB = 2 ** 30
SECTOR_SIZE = 32 * GIB
EPOCH_PER_DAY = 2880

class ArrayWrapperObj:
    def __init__(self, arr):
        self.arr = arr
    def get(self):
        return self.arr

2023-01-01
2023-02-12
2026-02-11


In [3]:
# plot a column from the trajectory df
def plt_x(cil_df_summary, 
          quantity_to_plot='day_pledge_per_QAP', 
          scale_by=EIB, 
          smooth_by=1):  
    plt.rcParams.update({'font.size': 18})
    plt.rcParams["figure.figsize"] = (8,5)

    colors_ = [['peachpuff', 'tan', 'peru'], 
               ['powderblue', 'skyblue', 'steelblue'], 
               ['lightsalmon', 'tomato', 'firebrick']]
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)][quantity_to_plot] / scale_by
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)][quantity_to_plot] / scale_by
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)][quantity_to_plot] / scale_by
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)][quantity_to_plot] / scale_by
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)][quantity_to_plot] / scale_by
    
    plt.fill_between(dates_, q05_t.rolling(smooth_by).median(), q95_t.rolling(smooth_by).median(), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    plt.fill_between(dates_, q25_t.rolling(smooth_by).median(), q75_t.rolling(smooth_by).median(), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    plt.plot(dates_, q50_t.rolling(smooth_by).median(), alpha=1, color=colors_[0][2], lw=2, label='Median')

    plt.xticks(rotation = 30)
    plt.title(f"{quantity_to_plot}")
    plt.legend(loc=2)
    plt.show()
    
# baseline crossing plots
def plt_blc(cil_df_summary, 
            quantity_to_plot='day_pledge_per_QAP', 
            scale_by=EIB, 
            smooth_by=1):
    # function to plot baseline crossing
    
    cil_df_summary = cil_df_summary[cil_df_summary['date'] <= pd.to_datetime(date(2024, 12, 1))]
    
    # define plot settings and palette
    plt.rcParams.update({'font.size': 18})
    plt.rcParams["figure.figsize"] = (18,18)
    colors_ = [['peachpuff', 'tan', 'peru'], 
               ['powderblue', 'skyblue', 'steelblue'], 
               ['lightsalmon', 'tomato', 'firebrick']]
    fig, axs = plt.subplots(4,2, constrained_layout=True)
    
    rolling_fn = lambda x: np.nanmedian(x)
    
    # select plotting data
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_RBP'] / scale_by
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_RBP'] / scale_by
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_RBP'] / scale_by
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_RBP'] / scale_by
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_RBP'] / scale_by
    # plot RBP predictions
    axs[0,0].fill_between(dates_, q5_t.rolling(smooth_by).apply(rolling_fn), q95_t.rolling(smooth_by).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='RBP Q90')
    axs[0,0].fill_between(dates_, q25_t.rolling(smooth_by).apply(rolling_fn), q75_t.rolling(smooth_by).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='RBP Q50')
    axs[0,0].plot(dates_, q50_t.rolling(smooth_by).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='RBP Median')
     # select plotting data
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_QAP'] / scale_by
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_QAP'] / scale_by
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_QAP'] / scale_by
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_QAP'] / scale_by
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_QAP'] / scale_by
# #     # plot QAP predictions
#     axs[0,0].fill_between(dates_, q5_t.rolling(smooth_by).median(), q95_t.rolling(smooth_by).median(), 
#                      alpha=0.5, color=colors_[1][0], label='QAP Q90')
#     axs[0,0].fill_between(dates_, q25_t.rolling(smooth_by).median(), q75_t.rolling(smooth_by).median(), 
#                      alpha=0.65, color=colors_[1][1], label='QAP Q50')
#     axs[0,0].plot(dates_, q50_t.rolling(smooth_by).median(), alpha=1, color=colors_[1][2], lw=2, label='QAP Median')
    
    # plot baseline
    axs[0,0].plot(dates_, baseline[0:len(dates_)], c='darkblue', label='baseline target')  
    axs[0,0].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    axs[0,0].tick_params(axis='x', labelrotation = 30)
    axs[0,0].legend(loc='upper right')
    axs[0,0].set_title('Baseline crossing')
    axs[0,0].set_ylabel('EiB')
    axs[0,0].set_ylim(10,20)
    axs[0,0].set_xlim(dates_[0],datetime.date(2023, 6, 6))
    
    # plot baseline crossing PDF
    axs[0,1].hist(date_cross_samples,
                bins=500,
                density=True,
                label='Baseline crossing PDF', color=colors_[0][0])
    cross_q50 = pd.Series(date_cross_samples).quantile(0.5, interpolation="midpoint")
    axs[0,1].axvline(cross_q50, color=colors_[1][2], linestyle='-', 
                   lw=2, alpha=1, label=f'Median {str(cross_q50)}')
    cross_q95 = pd.Series(date_cross_samples).quantile(0.95, interpolation="midpoint")
    axs[0,1].axvline(cross_q95, color=colors_[1][2], linestyle='-', 
                   lw=2, alpha=1, label=f'Q95 {str(cross_q95)}')
    cross_q5 = pd.Series(date_cross_samples).quantile(0.05, interpolation="midpoint")
    axs[0,1].axvline(cross_q5, color=colors_[1][2], linestyle='-', 
                   lw=2, alpha=1, label=f'Q5 {str(cross_q5)}')
#     axs[0,1].xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    axs[0,1].set_xlim(cross_q5-timedelta(days=30), cross_q95+timedelta(days=30))
    axs[0,1].tick_params(axis='x', labelrotation = 30)
    axs[0,1].set_title('Crossing empirical PDF')
    axs[0,1].legend(loc='upper right')

    # plot pledge per sector
#     w = 60
    w = 7
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['day_pledge_per_QAP'] 
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['day_pledge_per_QAP']
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['day_pledge_per_QAP']
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['day_pledge_per_QAP']
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['day_pledge_per_QAP']
    print(len(dates_), len(q05_t), len(q25_t), len(q50_t), len(q75_t), len(q95_t))
    print(len(q05_t.rolling(w).apply(rolling_fn)),
          len(q25_t.rolling(w).apply(rolling_fn)),
          len(q50_t.rolling(w).apply(rolling_fn)),
          len(q75_t.rolling(w).apply(rolling_fn)),
          len(q95_t.rolling(w).apply(rolling_fn)))
    print(len(np.where(np.isnan(q05_t.rolling(w).apply(rolling_fn)))[0]),
          len(np.where(np.isnan(q25_t.rolling(w).apply(rolling_fn)))[0]),
          len(np.where(np.isnan(q50_t.rolling(w).apply(rolling_fn)))[0]),
          len(np.where(np.isnan(q75_t.rolling(w).apply(rolling_fn)))[0]),
          len(np.where(np.isnan(q95_t.rolling(w).apply(rolling_fn)))[0]))
    axs[1,0].fill_between(dates_, q05_t.rolling(w).apply(rolling_fn), q95_t.rolling(w).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[1,0].fill_between(dates_, q25_t.rolling(w).apply(rolling_fn), q75_t.rolling(w).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[1,0].plot(dates_, q50_t.rolling(w).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[1,0].tick_params(axis='x', labelrotation = 30)
    axs[1,0].set_title('Pledge per 32 GiB')
    axs[1,0].legend(loc='upper right')
    axs[1,0].set_ylabel('FIL')
    axs[1,0].set_ylim(0,0.5)
    
    # plot network reward
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['day_network_reward'] 
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['day_network_reward'] 
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['day_network_reward'] 
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['day_network_reward']
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['day_network_reward']
    axs[1,1].fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[1,1].fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[1,1].plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[1,1].tick_params(axis='x', labelrotation = 30)
    axs[1,1].set_title('Minting')
    axs[1,1].legend(loc='upper right')
    axs[1,1].set_ylabel('FIL/day')
    
    # plot roi
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['1y_sector_roi'] * 100
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['1y_sector_roi']  * 100
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['1y_sector_roi'] * 100
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['1y_sector_roi'] * 100
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['1y_sector_roi'] * 100
    axs[2,1].fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[2,1].fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[2,1].plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[2,1].tick_params(axis='x', labelrotation = 30)
    axs[2,1].set_title('1y sector ROI')
    axs[2,1].legend(loc='upper right')
    axs[2,1].set_ylabel('%')
    axs[2,1].set_ylim(0,)
    xt = axs[2,1].get_xticks()
    xl = axs[2,1].get_xlim()
    
    # plot daily locked pledge
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['day_locked_pledge'] 
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['day_locked_pledge'] 
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['day_locked_pledge'] 
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['day_locked_pledge']
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['day_locked_pledge']
    axs[2,0].fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[2,0].fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).median(), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[2,0].plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[2,0].tick_params(axis='x', labelrotation = 30)
    axs[2,0].set_title('Daily locked pledge')
    axs[2,0].legend(loc='upper right')
    axs[2,0].set_ylabel('FIL/day')
    
    # plot network locked
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_locked'] 
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_locked'] 
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_locked'] 
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_locked']
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_locked']
    axs[3,0].fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[3,0].fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[3,0].plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[3,0].tick_params(axis='x', labelrotation = 30)
    axs[3,0].set_title('Network locked')
    axs[3,0].legend(loc='upper right')
    axs[3,0].set_ylabel('FIL')
    
    # plot lock / supply
    cil_df_summary['locked_over_supply'] = cil_df_summary['network_locked'] / cil_df_summary['circ_supply']
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['locked_over_supply'] * 100
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['locked_over_supply'] * 100
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['locked_over_supply'] * 100
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['locked_over_supply'] * 100
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['locked_over_supply'] * 100
    axs[3,1].fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    axs[3,1].fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    axs[3,1].plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    axs[3,1].tick_params(axis='x', labelrotation = 30)
    axs[3,1].set_title('Locked / Supply')
    axs[3,1].legend(loc='upper right')
    axs[3,1].set_ylabel('%')
    
    plt.show()

In [14]:
# simulation
# seperating from the expensive MCMC loop
def preprocess_expirations():
    # expirations
    res = get_sector_expiration_stats(start_date, current_date,end_date)

    rb_known_scheduled_expire_vec = res[0]
    qa_known_scheduled_expire_vec = res[1]
    known_scheduled_pledge_release_full_vec = res[2]

    fil_stats_df = get_historical_network_stats(start_date,current_date,end_date)

    current_day_stats = fil_stats_df[fil_stats_df["date"] >= current_date].iloc[0]

    rb_power_zero = current_day_stats["total_raw_power_eib"] * 1024.0
    qa_power_zero = current_day_stats["total_qa_power_eib"] * 1024.0
    
    return rb_known_scheduled_expire_vec, qa_known_scheduled_expire_vec, known_scheduled_pledge_release_full_vec, rb_power_zero, qa_power_zero, fil_stats_df

# def get_historical_renewal_rate(current_date,start_date)->np.array:
#     df_renewal_rate_orig = pd.read_csv('./offline_info/Scheduled_Expiration_by_Date_Breakdown_in_FIL.csv', 
#                                        parse_dates=['stateTime'])
#     df_renewal_rate = df_renewal_rate_orig.copy()
#     df_renewal_rate['Renewal_Rate'] = df_renewal_rate['Extend'] / (df_renewal_rate['Extend'] 
#                                  + df_renewal_rate['Expired']
#                                  + df_renewal_rate['Potential Expire'])
#     df_renewal_rate = df_renewal_rate[df_renewal_rate.stateTime <= str(current_date)]
#     df_renewal_rate = df_renewal_rate[df_renewal_rate.stateTime >= str(start_date)]
#     historical_renewal_rate = df_renewal_rate.Renewal_Rate.values
#     return historical_renewal_rate

def clip_all_powers(df_in):
    """
    'onboarded_power', 'cum_onboarded_power',
    'expire_scheduled_power', 'cum_expire_scheduled_power', 'renewed_power',
    'cum_renewed_power', 'total_power', 'power_type', 'total_qa_power_eib'
    """
    df_out = df_in.copy()
    for c in df_out.columns:
        if 'power' in c and c != 'power_type':
            df_out[c] = df_out[c].clip(lower=0)
    return df_out

def run_sim(fil_plus_rate:Union[float, np.array], 
            rb_onboard_power:[float, np.array], 
            renewal_rate:[float, np.array])->pd.DataFrame():
    future_renewal_rate = scalar_or_vector_to_vector(renewal_rate, forecast_lenght)
    renewal_rate_vec = np.concatenate([historical_renewal_rate,future_renewal_rate])
    
    rb_power_df, qa_power_df = forecast_power_stats(
        rb_power_zero,
        qa_power_zero,
        rb_onboard_power,
        rb_known_scheduled_expire_vec,
        qa_known_scheduled_expire_vec,
        future_renewal_rate,
        fil_plus_rate,
        duration,
        forecast_lenght,
        qap_method='basic'
    )
    
    ########## BUG FIX
    rb_power_df = clip_all_powers(rb_power_df)
    qa_power_df = clip_all_powers(qa_power_df)
    ##########
    rb_power_df["total_raw_power_eib"] = rb_power_df["total_power"]/1024.0
    qa_power_df["total_qa_power_eib"] = qa_power_df["total_power"]/1024.0

    power_df = build_full_power_stats_df(
        fil_stats_df,
        rb_power_df,
        qa_power_df,
        start_date,
        current_date,
        end_date,
    )

    rb_total_power_eib = power_df["total_raw_power_eib"].values
    qa_total_power_eib = power_df["total_qa_power_eib"].values
    qa_day_onboarded_power_pib = power_df["day_onboarded_qa_power_pib"].values
    qa_day_renewed_power_pib = power_df["day_renewed_qa_power_pib"].values
    
    # minting 
    mint_df = compute_minting_trajectory_df(
        start_date,
        end_date,
        rb_total_power_eib,
        qa_total_power_eib,
        qa_day_onboarded_power_pib,
        qa_day_renewed_power_pib,
    )

    start_day_stats = fil_stats_df.iloc[0]
    circ_supply_zero = start_day_stats["circulating_fil"]
    locked_fil_zero = start_day_stats["locked_fil"]
    burnt_fil_zero = start_day_stats["burnt_fil"]
    daily_burnt_fil = fil_stats_df["burnt_fil"].diff().mean()
    burnt_fil_vec = fil_stats_df["burnt_fil"].values

    cil_df = forecast_circulating_supply_df(
        start_date,
        current_date,
        end_date,
        circ_supply_zero,
        locked_fil_zero,
        daily_burnt_fil,
        duration,
        renewal_rate_vec,
        burnt_fil_vec,
        vest_df,
        mint_df,
        known_scheduled_pledge_release_full_vec
    )
    
    return cil_df

# sampling
def get_uniform_samples(central_estimate:float, 
                UR:float = 0.5, 
                n_samples:int = 100,
                key:int = 666,)->np.array:
    rng_key = random.split(random.PRNGKey(key))
    samples = dist.Uniform(central_estimate*(1-UR),
                           central_estimate*(1+UR)).expand([n_samples]).sample(rng_key[0])
    return np.array(samples)

def generate_prior_samples(n_samples=10, UR=0.5, 
                           fil_plus_rate_central=0.25, 
                           rb_onboard_power_central=8, 
                           renewal_rate_central=0.5)->pd.DataFrame:
    fil_plus_rate_samples = get_uniform_samples(central_estimate=fil_plus_rate_central, 
                                        UR=UR, 
                                        n_samples=n_samples)
    rb_onboard_power_samples = get_uniform_samples(central_estimate=rb_onboard_power_central, 
                                        UR=UR, 
                                        n_samples=n_samples)
    renewal_rate_samples = get_uniform_samples(central_estimate=renewal_rate_central, 
                                        UR=UR, 
                                        n_samples=n_samples)
    samples_df = pd.DataFrame({'fil_plus_rate': fil_plus_rate_samples,
                               'rb_onboard_power': rb_onboard_power_samples,
                               'renewal_rate': renewal_rate_samples})
    return samples_df

def generate_mcmc_forecast_samples(train_start_date: datetime.date,
                                   train_end_date: datetime.date,
                                   forecast_length: int,
                                   num_warmup_mcmc: int = 500,
                                   num_samples_mcmc: int = 100,
                                   seasonality_mcmc: int = 1000,
                                   num_chains_mcmc: int = 2):
    print("Forecasting Onboarding Power")
    forecast_rb_date_vec, rb_onboard_power_pred, _, _ = \
        mcmc.forecast_rb_onboard_power(train_start_date, 
                                       train_end_date,
                                       forecast_length,
                                       num_warmup_mcmc = num_warmup_mcmc,
                                       num_samples_mcmc = num_samples_mcmc,
                                       seasonality_mcmc = seasonality_mcmc,
                                       num_chains_mcmc = num_chains_mcmc)
    print("Forecasting Renewal Rate")
    forecast_rr_date_vec, renewal_rate_pred, _ , _ = \
        mcmc.forecast_renewal_rate(train_start_date, 
                                   train_end_date,
                                   forecast_length,
                                   num_warmup_mcmc = num_warmup_mcmc,
                                   num_samples_mcmc = num_samples_mcmc,
                                   seasonality_mcmc = seasonality_mcmc,
                                   num_chains_mcmc = num_chains_mcmc)
    
    print("Forecasting FIL+ Rate")
    forecast_fpr_date_vec, filplus_rate_pred, _, _ = \
        mcmc.forecast_filplus_rate(train_start_date, 
                                   train_end_date,
                                   forecast_length,
                                   num_warmup_mcmc = num_warmup_mcmc,
                                   num_samples_mcmc = num_samples_mcmc,
                                   seasonality_mcmc = seasonality_mcmc,
                                   num_chains_mcmc = num_chains_mcmc)
    assert np.array_equal(forecast_rb_date_vec, forecast_rr_date_vec)
    assert np.array_equal(forecast_rr_date_vec, forecast_fpr_date_vec)
    return rb_onboard_power_pred, renewal_rate_pred, filplus_rate_pred

def run_sim_mc(samples_df)->pd.DataFrame:
    cil_df_i, cil_df = pd.DataFrame(), pd.DataFrame()
    for i in tqdm(samples_df.index):
        fil_plus_rate = samples_df['fil_plus_rate'].iloc[i]
        if isinstance(fil_plus_rate, ArrayWrapperObj):
            fil_plus_rate = fil_plus_rate.get()
        rb_onboard_power = samples_df['rb_onboard_power'].iloc[i]
        if isinstance(rb_onboard_power, ArrayWrapperObj):
            rb_onboard_power = rb_onboard_power.get()
        renewal_rate = samples_df['renewal_rate'].iloc[i]
        if isinstance(renewal_rate, ArrayWrapperObj):
            renewal_rate = renewal_rate.get()
        cil_df_i = run_sim(fil_plus_rate, rb_onboard_power, renewal_rate)

        cil_df_i['iter'] = i
        cil_df_i['fil_plus_rate'] = ArrayWrapperObj(fil_plus_rate)
        cil_df_i['rb_onboard_power'] = ArrayWrapperObj(rb_onboard_power)
        cil_df_i['renewal_rate'] = ArrayWrapperObj(renewal_rate)
        cil_df = cil_df.append(cil_df_i)

    cil_df.index = cil_df.date
    cil_df.drop('date',axis=1,inplace=True)
    return cil_df

# baseline storage function
def baseline_storage(epoch:float)->float:
    # Spec value:
    # BASELINE_B0 = 2.88888888 
    # The following value is required to match sentinel
    # to Reproduce the first baseline crossing in backtest.
    # It's also close to the value the starboard use: 2.77. 
    # !Someone should look more closely at this.
    BASELINE_B0 = 2.7636 
    BASELINE_R = np.log(2) / (2880*365) # 1_051_200 in eopchs
    EPOCH_PER_DAY = 2880
    return BASELINE_B0 * np.exp(BASELINE_R * epoch)

# summary statistics of trajectory dynamics
def get_df_summary(df)->pd.DataFrame:
    df_summary = df.groupby(['date']).quantile([0.05,0.25,0.5,0.75,0.95]).reset_index(level=[0,1])
    df_summary.rename(columns = {'level_1':'quantile'}, inplace = True)
    return df_summary

# find baseline crossings
def find_crossings(df:pd.DataFrame)->pd.DataFrame:
    xing = np.array([])
    for i in range(df.iter[-1]+1):
        df_i = df[df['iter'] == i]
        df_crossing = df_i[df_i.network_RBP.values/EIB < baseline]
        if len(df_crossing) > 0:
            xing_i = df_i[df_i.network_RBP.values/EIB < baseline].index[0]
            xing = np.append(xing,xing_i)
    return xing

# add ROI to trajectory
def add_generated_quantities(df:pd.DataFrame)->pd.DataFrame:
    # add ROI to trajectory df
    df['day_pledge_per_QAP'] = SECTOR_SIZE * (df['day_locked_pledge']-df['day_renewed_pledge'])/(df['day_onboarded_power_QAP'])
    df['day_rewards_per_sector'] = SECTOR_SIZE * df.day_network_reward / df.network_QAP
    df['1y_return_per_sector'] = df[['iter','day_rewards_per_sector']].groupby('iter').rolling(duration).sum().shift(-duration+1).values.flatten()
    df['1y_sector_roi'] = df['1y_return_per_sector'] / df['day_pledge_per_QAP']
    
    return df

In [17]:
# separate expensive preprocessing from MCMC loop
rb_known_scheduled_expire_vec, qa_known_scheduled_expire_vec, known_scheduled_pledge_release_full_vec, rb_power_zero, qa_power_zero, fil_stats_df = preprocess_expirations()
vest_df = compute_vesting_trajectory_df(start_date, end_date)
# historical_renewal_rate = get_historical_renewal_rate(current_date-timedelta(days=1),start_date)
_, historical_renewal_rate = u.get_historical_renewal_rate(start_date, current_date)

# Use parameters distributions fitted from from data

In [ ]:
# from Distributions_to_parameterise_baseline_crossing_model.ipynb
model_samples_df = pd.read_csv('./offline_info/df_distributions_for_power_model_parameters.csv')
sns.pairplot(model_samples_df)

In [ ]:
model_samples_df.describe()

In [ ]:
# simulate trajectories for 
# warning this takes ~15min for 500 samples 
n_samples = 100
cil_df2 = run_sim_mc(model_samples_df[:n_samples])

# find baseine storage function and crossings
cil_df2_mean = cil_df2.groupby('date').mean()
baseline = baseline_storage(cil_df2_mean.days * EPOCH_PER_DAY)
date_cross_samples = find_crossings(cil_df2)

# add ROI & summary
cil_df2_roi = add_generated_quantities(cil_df2)
cil_df2_summary = get_df_summary(cil_df2_roi)

In [ ]:
plt_blc(cil_df2_summary, 
          quantity_to_plot='network_RBP', 
          scale_by=EIB)

# Test parameters sampled from uniform distribution for selected scenarios

## Example 1

In [ ]:
fil_plus_rate_central=0.25
rb_onboard_power_central=10
renewal_rate_central=0.5
UR=0.5 # % uncertainty about ^ assumptions
n_samples=10 # number of MC samples
prior_samples_df = generate_prior_samples(n_samples, 
                                          UR, 
                                          fil_plus_rate_central, 
                                          rb_onboard_power_central, 
                                          renewal_rate_central)

# simulate trajectories
cil_df = run_sim_mc(prior_samples_df)

# find baseine storage function and crossings
cil_df_mean = cil_df.groupby('date').mean()
baseline = baseline_storage(cil_df_mean.days * EPOCH_PER_DAY)
date_cross_samples = find_crossings(cil_df)

# add ROI & summary
cil_df_roi = add_generated_quantities(cil_df)
cil_df_summary = get_df_summary(cil_df_roi)

# plot
plt_blc(cil_df_summary, 
          quantity_to_plot='network_RBP', 
          scale_by=EIB)

## Example 2

In [ ]:
fil_plus_rate_central=0.25 # fraction between 0 and 1
rb_onboard_power_central=10 # PiB/day
renewal_rate_central=0.2 # fraction between 0 and 1
UR=0.5 # % uncertainty about ^ assumptions
n_samples=10 # number of MC samples
prior_samples_df = generate_prior_samples(n_samples, 
                                          UR, 
                                          fil_plus_rate_central, 
                                          rb_onboard_power_central, 
                                          renewal_rate_central)

# simulate trajectories
cil_df = run_sim_mc(prior_samples_df)

# find baseine storage function and crossings
cil_df_mean = cil_df.groupby('date').mean()
baseline = baseline_storage(cil_df_mean.days * EPOCH_PER_DAY)
date_cross_samples = find_crossings(cil_df)

# add ROI & summary
cil_df_roi = add_generated_quantities(cil_df)
cil_df_summary = get_df_summary(cil_df_roi)

# plot
plt_blc(cil_df_summary, 
          quantity_to_plot='network_RBP', 
          scale_by=EIB)

## Example 3

In [ ]:
fil_plus_rate_central=0.2
rb_onboard_power_central=1
renewal_rate_central=0.3
UR=0.005 # % uncertainty about ^ assumptions
n_samples=1 # number of MC samples
prior_samples_df = generate_prior_samples(n_samples, 
                                          UR, 
                                          fil_plus_rate_central, 
                                          rb_onboard_power_central, 
                                          renewal_rate_central)

# simulate trajectories
cil_df = run_sim_mc(prior_samples_df)

# find baseine storage function and crossings
cil_df_mean = cil_df.groupby('date').mean()
baseline = baseline_storage(cil_df_mean.days * EPOCH_PER_DAY)
date_cross_samples = find_crossings(cil_df)

# add ROI & summary
cil_df_roi = add_generated_quantities(cil_df)
cil_df_summary = get_df_summary(cil_df_roi)

# plot
plt_blc(cil_df_summary, 
          quantity_to_plot='network_RBP', 
          scale_by=EIB)

In [ ]:
cil_df_summary[(cil_df_summary['date'] == datetime.date(2022, 11, 20)) & (cil_df_summary['quantile'] == 0.5)]

In [6]:
# MCMC forecasting configuration
mcmc_train_len_days = 180
mcmc_train_start_date = today - datetime.timedelta(days=(DATA_LAG_DAYS + mcmc_train_len_days))
mcmc_train_end_date = mcmc_train_start_date + datetime.timedelta(days=mcmc_train_len_days)
num_warmup_mcmc = 5000
num_samples_mcmc = 25
seasonality_mcmc = 2000
num_chains_mcmc = 2

rb_onboard_power_pred, renewal_rate_pred, filplus_rate_pred = \
    generate_mcmc_forecast_samples(mcmc_train_start_date,
                                   mcmc_train_end_date,
                                   forecast_lenght,
                                   num_warmup_mcmc,
                                   num_samples_mcmc,
                                   seasonality_mcmc,
                                   num_chains_mcmc)

Forecasting Onboarding Power


  0%|          | 0/5025 [00:00<?, ?it/s]

  0%|          | 0/5025 [00:00<?, ?it/s]


                      mean       std    median      5.0%     95.0%     n_eff     r_hat
      coef_trend     -0.03      0.04     -0.03     -0.10      0.04     16.91      1.14
       init_s[0]     -1.16      8.47      0.59    -22.90      6.98     47.84      1.01
       init_s[1]      0.31      0.41      0.34     -0.31      0.98     55.70      0.98
       init_s[2]      0.17      0.68      0.13     -0.35      1.70      6.60      1.13
       init_s[3]     -0.23      0.44     -0.22     -1.12      0.34     51.18      0.96
       init_s[4]      0.43      0.57      0.52     -0.57      1.16     16.20      1.13
       init_s[5]     -0.32      0.45     -0.27     -1.24      0.25     43.77      1.02
       init_s[6]     -0.09      0.47     -0.13     -0.85      0.63     33.16      0.98
       init_s[7]     -0.10      0.45     -0.09     -0.86      0.51     29.67      1.03
       init_s[8]     -0.24      0.40     -0.29     -0.84      0.40     33.63      1.08
       init_s[9]      0.23      0.56      

Forecasting Renewal Rate


...

  0%|          | 0/5025 [00:00<?, ?it/s]


                      mean       std    median      5.0%     95.0%     n_eff     r_hat
      coef_trend      0.02      0.02      0.02     -0.01      0.05     58.60      1.04
       init_s[0]      0.17      0.65      0.01     -0.59      0.83     49.21      1.05
       init_s[1]      0.16      0.13      0.18     -0.01      0.34     19.60      1.03
       init_s[2]      0.37      0.26      0.41     -0.09      0.66     14.98      1.13
       init_s[3]      0.15      0.16      0.14     -0.06      0.40     46.74      1.03
       init_s[4]      0.13      0.15      0.14     -0.19      0.29     39.68      0.98
       init_s[5]      0.24      0.13      0.25      0.01      0.41     40.48      1.00
       init_s[6]      0.32      0.16      0.30      0.10      0.59     38.11      1.02
       init_s[7]      0.08      0.16      0.05     -0.08      0.45     37.69      0.99
       init_s[8]      0.52      0.15      0.51      0.29      0.74     38.98      1.03
       init_s[9]      0.25      0.14      

  0%|          | 0/5025 [00:00<?, ?it/s]

  0%|          | 0/5025 [00:00<?, ?it/s]


                      mean       std    median      5.0%     95.0%     n_eff     r_hat
      coef_trend     -0.02      0.02     -0.02     -0.07     -0.00     31.67      1.05
       init_s[0]      0.69     16.54      0.03    -29.84     18.17     56.53      0.98
       init_s[1]      0.15      0.37      0.18     -0.57      0.60     32.21      1.01
       init_s[2]     -0.32      0.61     -0.26     -0.86      0.49      3.92      1.15
       init_s[3]     -0.45      0.51     -0.52     -1.11      0.46     39.09      0.97
       init_s[4]      0.40      0.68      0.29     -0.73      1.31     40.23      1.00
       init_s[5]     -0.66      0.63     -0.69     -1.46      0.46     46.19      1.00
       init_s[6]     -0.67      0.60     -0.67     -1.46      0.24     58.53      0.98
       init_s[7]     -0.44      0.60     -0.49     -1.26      0.58     36.95      1.00
       init_s[8]     -0.86      0.65     -0.88     -1.82      0.04     30.61      1.00
       init_s[9]     -0.08      0.69     -

In [19]:
df_mcmc = pd.DataFrame(columns=['rb_onboard_power', 'renewal_rate', 'fil_plus_rate'])
df_mcmc['rb_onboard_power'] = df_mcmc['rb_onboard_power'].astype(object)
df_mcmc['renewal_rate'] = df_mcmc['renewal_rate'].astype(object)
df_mcmc['fil_plus_rate'] = df_mcmc['fil_plus_rate'].astype(object)
num_mcmc_samps = rb_onboard_power_pred.shape[0]
for i in range(num_mcmc_samps):
    df_row = pd.DataFrame({
        'rb_onboard_power': ArrayWrapperObj(np.asarray(rb_onboard_power_pred[i,:])),
        'renewal_rate': ArrayWrapperObj(np.asarray(renewal_rate_pred[i,:])),
        'fil_plus_rate': ArrayWrapperObj(np.asarray(filplus_rate_pred[i,:]))
#         'fil_plus_rate': ArrayWrapperObj(np.ones(forecast_lenght)*0.1)
    }, index=[0])
    df_mcmc = pd.concat([df_mcmc, df_row], ignore_index=True)

In [20]:
cil_mcmc_df = run_sim_mc(df_mcmc)

  0%|          | 0/50 [00:00<?, ?it/s]

In [ ]:
# find baseine storage function and crossings
cil_mcmc_mean = cil_mcmc_df.groupby('date').mean()
baseline = baseline_storage(cil_mcmc_mean.days * EPOCH_PER_DAY)
date_cross_samples = find_crossings(cil_mcmc_df)

# add ROI & summary
cil_mcmc_df_roi = add_generated_quantities(cil_mcmc_df)
cil_mcmc_df_summary = get_df_summary(cil_mcmc_df_roi)

In [ ]:
plt_blc(cil_mcmc_df_summary, 
          quantity_to_plot='network_RBP', 
          scale_by=EIB)

In [ ]:
def save_blc_plot(cil_df_summary, 
                  quantity_to_plot='day_pledge_per_QAP', 
                  scale_by=EIB, 
                  smooth_by=1):
    # function to plot baseline crossing
    
    # define plot settings and palette
    plt.rcParams.update({'font.size': 18})
    plt.rcParams["figure.figsize"] = (18,18)
    colors_ = [['peachpuff', 'tan', 'peru'], 
               ['powderblue', 'skyblue', 'steelblue'], 
               ['lightsalmon', 'tomato', 'firebrick']]
#     fig, axs = plt.subplots(4,2, constrained_layout=True)

    plt.figure(figsize=(10,4))
    
    rolling_fn = lambda x: np.nanmedian(x)
    
    # select plotting data
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_RBP'] / scale_by
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_RBP'] / scale_by
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_RBP'] / scale_by
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_RBP'] / scale_by
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_RBP'] / scale_by
    # plot RBP predictions
    plt.fill_between(dates_, q5_t.rolling(smooth_by).apply(rolling_fn), q95_t.rolling(smooth_by).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='RBP Q90')
    plt.fill_between(dates_, q25_t.rolling(smooth_by).apply(rolling_fn), q75_t.rolling(smooth_by).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='RBP Q50')
    plt.plot(dates_, q50_t.rolling(smooth_by).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='RBP Median')
     # select plotting data
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_QAP'] / scale_by
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_QAP'] / scale_by
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_QAP'] / scale_by
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_QAP'] / scale_by
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_QAP'] / scale_by
    
    # plot baseline
    plt.plot(dates_, baseline, c='darkblue', label='baseline target')  
    ax = plt.gca()
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
    ax.tick_params(axis='x', labelrotation = 30)
    plt.legend(loc='upper left', fontsize=11)
    ax.set_title('Baseline crossing')
    ax.set_ylabel('EiB')
    ax.set_ylim(10,20)
    ax.set_xlim(dates_[0],datetime.date(2023, 6, 6))
    
    plt.tight_layout()
    plt.savefig('/Users/kiran/Documents/hackmd/baseline_crossing/for_dave_blc.png')

save_blc_plot(cil_mcmc_df_summary, 
              quantity_to_plot='network_RBP', 
              scale_by=EIB)

In [ ]:
def save_roi_plot(cil_df_summary, 
                  quantity_to_plot='day_pledge_per_QAP', 
                  scale_by=EIB, 
                  smooth_by=1):
    # function to plot baseline crossing
    
    # define plot settings and palette
    plt.rcParams.update({'font.size': 18})
    plt.rcParams["figure.figsize"] = (18,18)
    colors_ = [['peachpuff', 'tan', 'peru'], 
               ['powderblue', 'skyblue', 'steelblue'], 
               ['lightsalmon', 'tomato', 'firebrick']]
#     fig, axs = plt.subplots(4,2, constrained_layout=True)

    plt.figure(figsize=(10,4))
    
    rolling_fn = lambda x: np.nanmedian(x)
    
    # select plotting data
    dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
    q05_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['1y_sector_roi'] * 100
    q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['1y_sector_roi']  * 100
    q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['1y_sector_roi'] * 100
    q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['1y_sector_roi'] * 100
    q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['1y_sector_roi'] * 100
    plt.fill_between(dates_, q05_t.rolling(7).apply(rolling_fn), q95_t.rolling(7).apply(rolling_fn), 
                     alpha=0.5, color=colors_[0][0], label='Q5-Q95')
    plt.fill_between(dates_, q25_t.rolling(7).apply(rolling_fn), q75_t.rolling(7).apply(rolling_fn), 
                     alpha=0.65, color=colors_[0][1], label='Q25-Q75')
    plt.plot(dates_, q50_t.rolling(7).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='Median')
    plt.tick_params(axis='x', labelrotation = 30)
    plt.title('1y sector ROI')
    plt.legend(loc='upper left', fontsize=11)
    plt.ylabel('%')
#     plt.set_ylim(0,)
    
#     dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
#     q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_RBP'] / scale_by
#     q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_RBP'] / scale_by
#     q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_RBP'] / scale_by
#     q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_RBP'] / scale_by
#     q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_RBP'] / scale_by
#     # plot RBP predictions
#     plt.fill_between(dates_, q5_t.rolling(smooth_by).apply(rolling_fn), q95_t.rolling(smooth_by).apply(rolling_fn), 
#                      alpha=0.5, color=colors_[0][0], label='RBP Q90')
#     plt.fill_between(dates_, q25_t.rolling(smooth_by).apply(rolling_fn), q75_t.rolling(smooth_by).apply(rolling_fn), 
#                      alpha=0.65, color=colors_[0][1], label='RBP Q50')
#     plt.plot(dates_, q50_t.rolling(smooth_by).apply(rolling_fn), alpha=1, color=colors_[0][2], lw=2, label='RBP Median')
#      # select plotting data
#     dates_ = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['date']
#     q5_t = cil_df_summary[(cil_df_summary['quantile'] == 0.05)]['network_QAP'] / scale_by
#     q25_t = cil_df_summary[(cil_df_summary['quantile'] == 0.25)]['network_QAP'] / scale_by
#     q50_t = cil_df_summary[(cil_df_summary['quantile'] == 0.5)]['network_QAP'] / scale_by
#     q75_t = cil_df_summary[(cil_df_summary['quantile'] == 0.75)]['network_QAP'] / scale_by
#     q95_t = cil_df_summary[(cil_df_summary['quantile'] == 0.95)]['network_QAP'] / scale_by
    
#     # plot baseline
#     plt.plot(dates_, baseline, c='darkblue', label='baseline target')  
#     ax = plt.gca()
#     ax.xaxis.set_major_locator(mdates.MonthLocator(interval=1))
#     ax.tick_params(axis='x', labelrotation = 30)
#     plt.legend(loc='upper left', fontsize=11)
#     ax.set_title('Baseline crossing')
#     ax.set_ylabel('EiB')
#     ax.set_ylim(10,20)
#     ax.set_xlim(dates_[0],datetime.date(2023, 6, 6))
    
    plt.tight_layout()
    plt.savefig('/Users/kiran/Documents/hackmd/baseline_crossing/for_dave_roi.png')

save_roi_plot(cil_mcmc_df_summary, 
              quantity_to_plot='network_RBP', 
              scale_by=EIB)

In [ ]:
# estimate the rate of decrease in minting rate in the following zones:
#   now -> baseline crossing
#   baseline_crossing -> 1/2 forecast window
#   1/2 forecast window -> end of forecast
def find_minting_decrease_rates_zones(df:pd.DataFrame)->pd.DataFrame:
    z1_avg_grad_vec = []
    z2_avg_grad_vec = []
    z3_avg_grad_vec = []
    for i in range(df.iter[-1]+1):
        df_i = df[df['iter'] == i]
        
        minting_rate_i = df_i['day_network_reward']
        
        df_crossing = df_i[df_i.network_RBP.values/EIB < baseline]
        if len(df_crossing) > 0:
            xing_i = df_i[df_i.network_RBP.values/EIB < baseline].index[0]
        else:
            continue
        
        try:
            z1_t1 = today; z1_t2 = xing_i
            ii_start = np.where(t==z1_t1)[0][0]; ii_end = np.where(t==z1_t2)[0][0]
            z1_avg_grad = np.nanmean(np.gradient(minting_rate_i[ii_start:ii_end]))

            z2_t1 = z1_t2; days_remaining = (today + timedelta(days=forecast_lenght) - z2_t1).days
            z2_t2 = median_baseline_crossing + timedelta(days=days_remaining//2)
            ii_start = np.where(t==z2_t1)[0][0]; ii_end = np.where(t==z2_t2)[0][0]
            z2_avg_grad = np.nanmean(np.gradient(minting_rate_i[ii_start:ii_end]))

            z3_t1 = z2_t2
            ii_start = np.where(t==z3_t1)[0][0]
            z3_avg_grad = np.nanmean(np.gradient(minting_rate[ii_start:]))
        except:
            continue
        
        z1_avg_grad_vec.append(z1_avg_grad)
        z2_avg_grad_vec.append(z2_avg_grad)
        z3_avg_grad_vec.append(z3_avg_grad)
        
    return z1_avg_grad_vec, z2_avg_grad_vec, z3_avg_grad_vec

def find_minting_decrease_rates_Ymo(df:pd.DataFrame)->pd.DataFrame:
    """
    Finds the rate of change in minting rates for the same timeperiod (T)
    between today and baseline crossing, and baseline crossing + T
    Zone 1 --> [Today, Baseline Crossing]
    T = (baseline_crossing - today).days
    Zone 2 --> [Baseline Crossing, Baseline Crossing + T]
    Zone 3 --> [Baseline Crossing + T, End of Forecasting Window]
    """
    # generate the "no crossing" scenario as a hacky method of computing max minting rate
    optimistic_scenario_df = run_sim(np.ones(forecast_lenght)*0.99, 
                                     np.ones(forecast_lenght)*150,  # exceeds the historical max
                                     np.ones(forecast_lenght)*0.99)
    # sanity check that we don't have a baseline crossing w/ the optimistic scenario
    sanity_check_crossing = np.where(optimistic_scenario_df.network_RBP.values/EIB < baseline)[0]
    assert len(sanity_check_crossing) == 0

    max_minting_rate = optimistic_scenario_df['day_network_reward']
    optimistic_t = optimistic_scenario_df['date']

    z1_avg_grad_vec = []
    z1_maxminting_grad_vec = []
    z2_avg_grad_vec = []
    z2_maxminting_grad_vec = []
    z3_avg_grad_vec = []
    z3_maxminting_grad_vec = []
    days_to_blc_vec = []
    for i in range(df.iter[-1]+1):
        df_i = df[df['iter'] == i]
        
        t_i = df_i.index
        minting_rate_i = df_i['day_network_reward']
        # assert time-alignment so that we can use the same indices
        assert np.array_equal(pd.to_datetime(optimistic_scenario_df['date'].values), 
                              pd.to_datetime(t_i))
        
        df_crossing = df_i[df_i.network_RBP.values/EIB < baseline]
        if len(df_crossing) > 0:
            xing_i = df_i[df_i.network_RBP.values/EIB < baseline].index[0]
        else:
            continue
        
        try:
            z1_t1 = today; z1_t2 = xing_i
            ii_start = np.where(t_i==z1_t1)[0][0]; ii_end = np.where(t_i==z1_t2)[0][0]
            z1_avg_grad = np.nanmean(np.gradient(minting_rate_i[ii_start:ii_end]))
            max_minting_rate_zone1_avg_grad = np.nanmean(np.gradient(max_minting_rate[ii_start:ii_end]))
            
            days_today_to_blc = (z1_t2 - z1_t1).days
            z2_t1 = z1_t2; z2_t2 = z2_t1 + timedelta(days=days_today_to_blc)
            ii_start = np.where(t_i==z2_t1)[0][0]; ii_end = np.where(t_i==z2_t2)[0][0]
            z2_avg_grad = np.nanmean(np.gradient(minting_rate_i[ii_start:ii_end]))
            max_minting_rate_zone2_avg_grad = np.nanmean(np.gradient(max_minting_rate[ii_start:ii_end]))
            
            z3_t1 = z2_t2
            ii_start = np.where(t_i==z3_t1)[0][0]
            z3_avg_grad = np.nanmean(np.gradient(minting_rate_i[ii_start:]))
            max_minting_rate_zone3_avg_grad = np.nanmean(np.gradient(max_minting_rate[ii_start:]))
        except:
            continue
        
        z1_avg_grad_vec.append(z1_avg_grad)
        z2_avg_grad_vec.append(z2_avg_grad)
        z3_avg_grad_vec.append(z3_avg_grad)
        z1_maxminting_grad_vec.append(max_minting_rate_zone1_avg_grad)
        z2_maxminting_grad_vec.append(max_minting_rate_zone2_avg_grad)
        z3_maxminting_grad_vec.append(max_minting_rate_zone3_avg_grad)
        days_to_blc_vec.append(days_today_to_blc)
        
    return z1_avg_grad_vec, z2_avg_grad_vec, z3_avg_grad_vec, days_to_blc_vec, \
            z1_maxminting_grad_vec, z2_maxminting_grad_vec, z3_maxminting_grad_vec

# z1_avg_grad_vec, z2_avg_grad_vec, z3_avg_grad_vec = find_minting_decrease_rates_zones(cil_mcmc_df)
z1_avg_grad_vec, z2_avg_grad_vec, z3_avg_grad_vec, days_to_blc_vec, \
    z1_maxminting_grad_vec, z2_maxminting_grad_vec, z3_maxminting_grad_vec = \
    find_minting_decrease_rates_Ymo(cil_mcmc_df)

In [ ]:
plt.figure(figsize=(8,4))
plt.subplot(1,3,1)
plt.hist(z1_avg_grad_vec, label='Zone-1[$\mu=%0.02f$]' % (np.mean(z1_avg_grad_vec,)))
plt.hist(z1_maxminting_grad_vec, label='(MAX)Zone-1[$\mu=%0.02f$]' % (np.mean(z1_maxminting_grad_vec,)))
plt.xticks(fontsize=12); plt.yticks(fontsize=12); 
plt.title('Zone-1\nActual/Max=%0.02f' % (np.mean(z1_avg_grad_vec)/np.mean(z1_maxminting_grad_vec), ), fontsize=12)
plt.subplot(1,3,2)
plt.hist(z2_avg_grad_vec, label='Actual')
plt.hist(z2_maxminting_grad_vec, label='MAX')
plt.xticks(fontsize=12); plt.yticks(fontsize=12); 
plt.title('Zone-2\nActual/Max=%0.02f' % (np.mean(z2_avg_grad_vec)/np.mean(z2_maxminting_grad_vec), ), fontsize=12)
plt.xlabel('Minting Rate', fontsize=12)
plt.legend(prop={'size': 12})
plt.subplot(1,3,3)
plt.hist(z3_avg_grad_vec, label='Zone-3[$\mu=%0.02f$]' % (np.mean(z3_avg_grad_vec,)))
plt.hist(z3_maxminting_grad_vec, label='(MAX)Zone-3[$\mu=%0.02f$]' % (np.mean(z3_maxminting_grad_vec,)))
plt.xticks(fontsize=12); plt.yticks(fontsize=12); 
plt.title('Zone-3\nActual/Max=%0.02f' % (np.mean(z3_avg_grad_vec)/np.mean(z3_maxminting_grad_vec), ), fontsize=12)
# plt.legend(prop={'size': 10})
plt.tight_layout()

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(days_to_blc_vec, label='$\mu=%0.02f$' % (np.mean(days_to_blc_vec),))
plt.title('Days to Baseline Crossing')
plt.legend(prop={'size': 14})